In [1]:
# import pandas as pd
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import control as ct

In [3]:
#run code in cell below before running this one and then run the next cell again
%matplotlib

Using matplotlib backend: <object object at 0x0000023371383A50>


In [9]:
#model response
m = 0.023
R = 0.02
g = 9.8
L = 0.20
d = 0.12
J = (2/3)*m*R**2
s = ct.tf('s')
G = ((m*g*d)/(L*((J/R**2)+m)))*(1/s**2)

Kp = 3
Ki = 0.3
Kd = 4

C = Kp+Ki/s+Kd*s
plant = ct.feedback(G*C)
T,U = ct.step_response(15*plant,T = 5)
fig8,ax8 = plt.subplots()

L = len(U)
sett = []
for i in range(0,L):
    if (U[-1]-U[i])/U[-1]<0.05:
        I = i
        sett=T[i]
        Usett = U[i]
        break
dict = ct.step_info(15*plant)
print(dict)
over = dict['Overshoot']
print('Overshoot = ',over,'%')
rise = dict['RiseTime']
print('Rise time =',rise,'s')
print("Settling time =",sett,'s')

ax8.plot(T,U,label = 'response')
ax8.axhline(15,color = 'red',linestyle ='dotted',label = 'setpoint')
ax8.axhline(U[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax8.axhline(U[-1]*1.05,color = 'green',linestyle = 'dotted',label = '5% upper')
ax8.grid()
ax8.plot(sett,Usett,'*',label = "settling point",color = 'r')
ax8.vlines(T[np.argmax(U)],ymin=15,ymax=np.max(U),label = 'Overshoot', color = 'red')
#ax8.vlines(0.45,ymin=15,ymax = U[-1],color='r',label = "steady state error")
ax8.legend()
ax8.set_ylabel('Ball Position [cm]',fontsize=15)
ax8.set_xlabel('Time [s]',fontsize=15)
fig8.suptitle('Trial-and-Error Tuned Model Response for Squash Ball',fontsize=15)

{'RiseTime': 0.1361614362376432, 'SettlingTime': nan, 'SettlingMin': 13.587238028709084, 'SettlingMax': 15.646820364985247, 'Overshoot': 4.3121357665683275, 'Undershoot': 0, 'Peak': 15.646820364985247, 'PeakTime': 0.4608540918812539, 'SteadyStateValue': 14.999999999999998}
Overshoot =  4.3121357665683275 %
Rise time = 0.1361614362376432 s
Settling time = 0.17598343685300208 s


Text(0.5, 0.98, 'Trial-and-Error Tuned Model Response for Squash Ball')

In [10]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figraw,axsraw = plt.subplots(5,1,figsize=(6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)
figz,axz = plt.subplots(5,1,figsize = (6,9))
plt.subplots_adjust(top = 0.95,bottom = 0.05)

fig.suptitle('Individual Trial-and-Error Tuned Responses for Squash Ball',fontsize=15)
figraw.suptitle('Unfiltered Trial-and-Error Tuned Responses for Squash Ball',fontsize=15)
figz.suptitle('Motor Angle for Trial-and-Error Tuned Squash Ball Response',fontsize=15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Trial-and-Error Tuned Squash Ball Responses',fontsize=15)
#fig3.suptitle('Unfiltered Average of Trial-and-Error Tuned Squash Ball Responses')
ax2.set_xlabel('Time [s]',fontsize=15)
ax2.set_ylabel('Distance [cm]',fontsize=15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 600, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 761
for i in range(1,6):
    df = pd.read_csv('BBS_PID_{}.csv'.format(i+1))
    
    time = (df['time'])
    print(len(time))
    time = time[0:END]
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs[i-1].plot(time,filtered,label = 'filtered',color = 'b')
    axs[i-1].plot(time,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs[i-1].set_xlabel('Time [s]')
    axs[i-1].set_ylabel('Distance [cm]')
    axs[i-1].axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
    axs[i-1].grid()
    
    axsraw[i-1].plot(time,distance)
    axsraw[i-1].set_xlabel('Time [s]')
    axsraw[i-1].set_ylabel('Distance [cm]')
    axsraw[i-1].grid()
    
    axz[i-1].plot(time,angle_filt,label = 'filtered',color = 'b')
    axz[i-1].plot(time,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz[i-1].set_xlabel('Time [s]')
    axz[i-1].set_ylabel('Servo Angle [degrees]')
    axz[i-1].grid()
    axz[i-1].axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')
    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
axs[0].legend()
axz[0].legend()
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)

filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
OS = np.max(filtered)
OSind = np.argmax(filtered)
OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax2.axhline(filtered[-1]*1.05,color = 'g',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid(which='both')
ax2.plot(sett,Usett,'p',label = "settling point",color = 'r')
ax2.axhline(filtered[-1],color='b',label = "steady state error",alpha=0.6,linestyle = 'dotted')
ax2.vlines(OST,15,OS,color = 'r',label = 'overshoot')
ticks = np.linspace(0,25,26)
ax2.set_yticks(ticks)
ax2.legend()
#ax3.plot(time_ave,dist_ave,label = 'response')
print('Overshoot=',((np.max(filtered)-filtered[-1])/filtered[-1])*100)
print('Settle =',(sett))
print("SSE=", (filtered[-1]-15)/0.15)

830
761
1110
1429
1658
Overshoot= 42.214606175692346
Settle = 5.1015999999999995
SSE= 1.376057029362651


In [13]:
#EXPERIMENTAL RESPONSE

#SETTING UP PLOTS
fig,axs = plt.subplots()

figraw,axsraw = plt.subplots()

figz,axz = plt.subplots()


fig.suptitle('Individual Trial-and-Error Tuned Responses for Squash Ball',fontsize=15)
figraw.suptitle('Unfiltered Trial-and-Error Tuned Responses for Squash Ball',fontsize=15)
figz.suptitle('Motor Angle for Trial-and-Error Tuned Squash Ball Response',fontsize=15)

fig2,ax2 = plt.subplots()
#fig3,ax3 = plt.subplots()
fig2.suptitle('Average of Trial-and-Error Tuned Squash Ball Responses',fontsize=15)
#fig3.suptitle('Unfiltered Average of Trial-and-Error Tuned Squash Ball Responses')
ax2.set_xlabel('Time [s]',fontsize=15)
ax2.set_ylabel('Distance [cm]',fontsize=15)
#ax3.set_xlabel('Time [s]')
#ax3.set_ylabel('Distance [cm]')

#Create Lists for Inidividual Subplots
time_1 = []
time_2 = []
time_3 = []
time_4 = []
time_5 = []
distance_1 = []
distance_2 = []
distance_3 = []
distance_4 = []
distance_5 = []
angle_1 = []
angle_2 = []
angle_3 = []
angle_4 = []
angle_5 = []
time_ave = []
dist_ave = []

#Creating Butterworth Filter
#scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba', fs=None)
sos = signal.butter(2, 600, 'lp', fs=50000, output='sos')

#NB NB NB RUN THIS CELL ONCE TO SEE THE LENGTHS OF THE DIFFERENT DATASETS AND THEN SET THE VALUE OF "END" TO THE SHORTEST ONE
#CHANGE NAME OF CSV TO DATASET BEING USED
END = 761
for i in range(1,6):
    df = pd.read_csv('BBS_PID_{}.csv'.format(i+1))
    
    time = (df['time'])
    print(len(time))
    time = time[0:END]
    distance = df['distance']
    distance = distance[0:END]
    angle = df['angle']-87
    angle = angle[0:END]
    filtered = signal.sosfiltfilt(sos,distance)
    angle_filt = signal.sosfiltfilt(sos,angle)
    
    axs.plot(time,filtered,label = 'Filtered {}'.format(i))
    #axs.plot(time,distance,alpha=0.4,label = 'unfiltered',color = 'darkslategrey')
    axs.set_xlabel('Time [s]',fontsize=15)
    axs.set_ylabel('Distance [cm]',fontsize=15)

    axs.grid()
    
    axsraw.plot(time,distance)
    axsraw.set_xlabel('Time [s]')
    axsraw.set_ylabel('Distance [cm]')
    axsraw.grid()
    
    axz.plot(time,angle_filt,label = 'Filtered {}'.format(i))
    #axz.plot(time,angle,alpha=0.35,label = 'unfiltered',color = 'darkslategrey')
    axz.set_xlabel('Time [s]',fontsize=15)
    axz.set_ylabel('Servo Angle [degrees]',fontsize=15)
    axz.grid()

    
    time_name =f"time_{i}"
    globals()[time_name]=((time[0:END].to_numpy()))
    distance_name = f"distance_{i}"
    globals()[distance_name]=distance[0:END].to_numpy()
    
axz.axhline(0,color = 'r',linestyle = 'dotted',label = '0 deg')    
axs.axhline(15,color = 'r',linestyle = 'dotted',label = 'Setpoint')    
axs.legend()
axz.legend()
for i in range(0,END):
    time_ave.append(sum([time_1[i],time_2[i],time_3[i],time_4[i],time_5[i]])/5)
    dist_ave.append(sum([distance_1[i],distance_2[i],distance_3[i],distance_4[i],distance_5[i]])/5)

filtered = signal.sosfiltfilt(sos,dist_ave)
L = len(filtered)
zeros = []
OS = np.max(filtered)
OSind = np.argmax(filtered)
OST = time_ave[OSind]

for i in range(0,L-1):
    if abs((filtered[-1]-filtered[i-1])/filtered[-1])>0.05 and abs((filtered[-1]-filtered[i+1])/filtered[-1])<0.05:
        zeros.append(time_ave[i])
for i in range(0,L):
    if (filtered[-1]-filtered[i])/filtered[-1]<0.05 and time_ave[i]>=zeros[-1]:
        I = i
        sett=time_ave[i]
        Usett = filtered[i]
        break
#Plotting Average Filtered Response with Overlayed Unfiltered Response
ax2.plot(time_ave,filtered,label = 'filtered',color = 'b')
ax2.plot(time_ave,dist_ave,alpha = 0.35,label = "unfiltered",color = 'darkslategrey')
ax2.axhline(filtered[-1]*0.95,color = 'orange',linestyle = 'dotted',label = '5% lower')
ax2.axhline(filtered[-1]*1.05,color = 'g',linestyle = 'dotted',label = '5% upper')
ax2.axhline(15,color = 'r',linestyle = 'dotted',label = 'setpoint')
ax2.grid(which='both')
ax2.plot(sett,Usett,'p',label = "settling point",color = 'r')
ax2.axhline(filtered[-1],color='b',label = "steady state error",alpha=0.6,linestyle = 'dotted')
ax2.vlines(OST,15,OS,color = 'r',label = 'overshoot')
ticks = np.linspace(0,25,26)
ax2.set_yticks(ticks)
ax2.legend()
#ax3.plot(time_ave,dist_ave,label = 'response')
print('Overshoot=',((np.max(filtered)-filtered[-1])/filtered[-1])*100)
print('Settle =',(sett))
print("SSE=", (filtered[-1]-15)/0.15)

830
761
1110
1429
1658
Overshoot= 42.214606175692346
Settle = 5.1015999999999995
SSE= 1.376057029362651
